# This Notebook provides data clening process and new features engineering.

The values in all columns are mapped to the one standart and unified if necessary.

Missing values are treated.

Inadequate values and categories with few observations are removed.

# Libraries imports

In [1]:
!pip install python-Levenshtein
!pip install pandas
!pip install datetime
!pip install fuzzywuzzy
!pip install copy
!pip install numpy

  Using cached python_Levenshtein-0.25.1-py3-none-any.whl.metadata (3.7 kB)
  Using cached Levenshtein-0.25.1-cp38-cp38-win_amd64.whl.metadata (3.4 kB)
  Using cached rapidfuzz-3.8.1-cp38-cp38-win_amd64.whl.metadata (11 kB)
Using cached python_Levenshtein-0.25.1-py3-none-any.whl (9.4 kB)
Using cached Levenshtein-0.25.1-cp38-cp38-win_amd64.whl (98 kB)
Using cached rapidfuzz-3.8.1-cp38-cp38-win_amd64.whl (1.7 MB)
  Using cached DateTime-5.5-py3-none-any.whl.metadata (33 kB)
  Using cached zope.interface-6.3-cp38-cp38-win_amd64.whl.metadata (43 kB)
Using cached DateTime-5.5-py3-none-any.whl (52 kB)
Using cached zope.interface-6.3-cp38-cp38-win_amd64.whl (204 kB)
  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl.metadata (4.9 kB)
Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl (18 kB)


ERROR: Could not find a version that satisfies the requirement copy (from versions: none)
ERROR: No matching distribution found for copy


In [5]:
import pandas as pd
from datetime import datetime
from fuzzywuzzy import fuzz,process
from copy import deepcopy
import numpy as np

# Auxilary functions

In [301]:
def get_distribution_column(column):
    """
    Get the distribution of not None values in column.

    Parameters
    ----------
    - column (pd Series): The Series  
    - column (str): column name in data to be cleaned
    - threshold (int): the smallest acceptable number of observations to leave the category

    Returns
    -------
    - pd Dateframe: A DataFrame with column values as Index, containing two columns: 'Values count'(shows the count for each index), 'Values percentage' (shows the percentage of data without Nones containing index)
    """
    values_counts = column.value_counts()
    values_percentage = (column.value_counts() /len(column.dropna())*100).round(2)
    return pd.DataFrame({'Values count': values_counts,'Values percentage': values_percentage})

def delete_column_categories(column,threshold,data):
    """
    Deletes the data observations containing column categories that make less observations that the threshold

    Parameters
    ----------
    - data (pd DataFrame): The DataFrame 
    - column (str): column name in data to be cleaned
    - threshold (int): the smallest acceptable number of observations to leave the category
    """
    column_distribution = get_distribution_column(data[column]) # get column categories distribution 
    values_to_delete = column_distribution[column_distribution['Values count'] < threshold].index # get categories that contain less observation than threshold
    mask = data[column].isin(values_to_delete) # get mask that sets True for indexes that will be deleted
    data_index_to_delete = data[mask].index # get data indexes of observations that will be deleted
    data.drop(data_index_to_delete, inplace=True) #remove data observations 
   


def get_ukr_unique(column_name,data):
    """
    Get unique values for each year.

    Parameters
    ----------
    - column_name (str): column name in data
    - data (pd DataFrame): The DataFrame 

    Returns
    -------
    - list: list of lists, each containing year`s unique values
    """
    years = data['Timestamp'].dropna().dt.year.unique() # get years that are present
    res = []
    for year in years:
        year_unique = data[ data['Timestamp'].dt.year == year][column_name].unique().tolist() # get unique values for year
        res.append(year_unique)
    return res


def get_intersaction_sets(lst):
    """
    Get intersection set of value for list of lists.

    Parameters
    ----------
    - lst (list): list of lists

    Returns
    -------
    - set: set of values that is present in each list
    """
    current = set(lst[0]) #make first element of list a set
    for i,s in enumerate(lst):
        if i!=0:
            current &= set(s) # find intersection with new list and set it as current
    return current

def find_similar(column_categories,score_cutoff=90):

    """
    Finds the similar subsets of column categories

    Parameters
    ----------
    - column_categories (array): array containing unique categories in column
    Returns
    -------
    - set: A set of frozensets, which are groups of similar values
    """

    similar_sets = []
    for i in column_categories:
        # find similar set
        similar_values_scores = process.extractBests(i,column_categories,score_cutoff=score_cutoff)
        similar_values = [j[0] for j in similar_values_scores]
        similar_set = frozenset([i] + similar_values)

        # check if there is a set that intersect the new one

        intersection = False
        for old_set in similar_sets:
            if similar_set.intersection(old_set):
                
                # update similar sets if there is an intersection
                intersection = True
                new_set = similar_set.union(old_set)
                similar_sets.remove(old_set)
                similar_sets.append(new_set)
                break
        
        # if intersections was not found add to similar sets      
        if not intersection:   
            similar_sets.append(similar_set)
    return set(similar_sets)

def make_map_from_similar_sets(column_categories,score_cutoff=90):
    """
    Forms a mapping dict for set of similar subsets of column categories, 
    by setting the longest value as key and other elements as values

    Parameters
    ----------
    - set: A set of frozensets, which are groups of similar values
    Returns
    -------
    - tuple: First element is mapping dict, second element is dict, where keys are values of single sets and values are empty lists
    """
    similar = find_similar(column_categories,score_cutoff=score_cutoff)
    undefined = {}
    mapping = {}
    for i in similar:
        values = list(i)
        if len(values)==1: # if there is only one element in set add it to undefined dict
            undefined[values[0]]=[]
        else:    # otherwise add the longest element as key to mappping and all other elements as corresponding value
            key = max(values,key = lambda x: len(x)) 
            values.remove(key)
            mapping[key] = values

    return mapping,undefined

def replace_spheres(sphere_map,sphere_df,spheres,reverse_dict = True):
    """
    Update spheres and get new sphere distribution
    Parameters
    ----------
    - spheres (pd.Series): A column in data containing lists of values
    - sphere_df (pd.Series): A Series object of flattened spheres
    - sphere_map (dict): A mapping dictionary, where keys are values that need to changes, and values are categories to what the keys need to be changed
    - reverse_dict (bool): When True the sphere_map keys and values are swapped
    Returns
    -------
    - tuple: First element is updated spheres, second element is distribution of updated spheres
    """
    def replace_df_lst(lst,sphere_map):
        return [sphere_map.get(i, i) for i in lst]
    if reverse_dict:
        sphere_map = {i:key for key,value in sphere_map.items() for i in value}
    sphere_df.replace(sphere_map,inplace=True)
    spheres = spheres.apply(replace_df_lst,sphere_map=sphere_map)
    sphere_distribution = get_distribution_column(sphere_df)
    return spheres,sphere_distribution

# Data reading

In [6]:
data = pd.read_csv('processed_data\data.csv',low_memory=False)
data.drop('Unnamed: 0',axis=1,inplace=True)

In [7]:
data

,Timestamp,Ваша основна зайнятість в ІТ зараз...,В якій області ви зараз живете?,Де ви працюєте?,Ваша стать,Ваш вік,Яка у вас освіта?,"Вкажіть ваш основний заклад вищої освіти (якщо вчилися в кількох – той, де провели найбільше часу)",Знання англійської мови,Спеціалізація,...,Вкажіть суму цього бонуса у після податків,Чи є у вас науковий ступінь?,Чи використовуєте ви у своїй роботі мови програмування (одну чи декілька)?,Загальний стаж роботи в ІТ,"Ставка у $$$ за годину, яку ви виставляєте клієнтам",Де ви зараз живете?,"Якщо ви працюєте в GameDev, оберіть вашу спеціалізацію",Вкажіть вашу спеціалізацію Game Design,Вкажіть вашу спеціалізацію Sound,Ваша спеціалізація Support
0,07/12/2020 13:06,"Так, працюю full-time",Київ,Віддалено,Чоловік,31,Вища,НТУУ «КПІ»,Upper-Intermediate,Support,...,4000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12/12/2020 11:33,"Так, працюю full-time",Київ,Віддалено,Чоловік,36,Вища,ХНУРЕ,Upper-Intermediate,"Software Engineer (Junior, Middle, Senior, Tea...",...,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,08/12/2020 20:21,"Так, працюю full-time",Київ,Віддалено,Чоловік,31,Вища,НТУУ «КПІ»,Advanced,"Software Engineer (Junior, Middle, Senior, Tea...",...,1100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,08/12/2020 12:45,"Так, працюю full-time",Київ,"Частково віддалено, частково в офісі",Чоловік,33,Вища,КНУБА,Intermediate,"Software Engineer (Junior, Middle, Senior, Tea...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,07/12/2020 10:47,"Так, працюю full-time",Дніпро,"Частково віддалено, частково в офісі",Чоловік,42,Вища,NaN,Upper-Intermediate,"Project/Product/Program Manager, Product Owner...",...,30000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77089,6.26.2023 13:27:47,Працюю full-time в ІТ-компанії чи ІТ-відділі,Київ чи область,NaN,Жінка,29,"Вища (бакалавр, спеціаліст, магістр) - одна аб...",NaN,Upper-Intermediate,HR/Recruiter/Learning and Development,...,NaN,NaN,NaN,NaN,NaN,В Україні,NaN,NaN,NaN,NaN
77090,6.26.2023 13:28:38,Працюю full-time в ІТ-компанії чи ІТ-відділі,Дніпро чи область,NaN,Жінка,21,"Вища (бакалавр, спеціаліст, магістр) - одна аб...",NaN,Upper-Intermediate,"Designer / Artist (Graphic, Video, UX/UI)",...,NaN,NaN,NaN,NaN,NaN,В Україні,NaN,NaN,NaN,NaN
77091,6.26.2023 13:42:22,Працюю full-time в ІТ-компанії чи ІТ-відділі,Львів чи область,NaN,Чоловік,25,Середня,NaN,Intermediate,"QA / AQA Engineer (Junior, Middle, Senior, Tea...",...,NaN,NaN,"Ні, не використовую",NaN,NaN,В Україні,NaN,NaN,NaN,NaN
77092,6.26.2023 13:48:00,Працюю full-time в ІТ-компанії чи ІТ-відділі,Полтава чи область,NaN,Чоловік,28,"Вища (бакалавр, спеціаліст, магістр) - одна аб...",NaN,Pre-Intermediate,Software Engineer,...,NaN,NaN,"Так, використовую",NaN,NaN,В Україні,NaN,NaN,NaN,NaN


Translate columns name to English

In [303]:
columns_translation_dict = {'В якій області ви зараз живете?': 'location',
 'Ваша стать': 'gender',
 'Ваш вік': 'age',
 'Яка у вас освіта?':'education',
 'Знання англійської мови': 'english_level',
 'Спеціалізація': 'specialisation',
 'Ваш тайтл на цій посаді': 'job_level',
 'Вкажіть вашу основну спеціалізацію': 'job_type',
 'Основна мова програмування': 'programming_language',
 'Вкажіть вашу спеціалізацію - QA':'qa_position',
 'Оберіть вашу посаду Designer / Artist':'designer_position',
 'Ваша посада Marketing':'marketing_position',
 'Оберіть вашу посаду HR':'hr_position',
 'Ваша посада Management':'management_position',
 'Ваша посада Analyst':'analyst_position',
 'Ваша посада Data Science':'data_scientist_position',
 'Ваша посада DevOps, SRE':'devops_position',
 'Ваша посада CTO, Director':'executive_position',
 'В якій сфері проєкт, в якому ви зараз працюєте?':'project_sphere',
 'Основний напрям роботи компанії, в якій працюєте':'company_type',
 'Загальний стаж роботи за нинішньою ІТ-спеціальністю':'experience',
 'Зарплата / дохід у $$$ за місяць, лише ставка після сплати податків':'salary',
 'Ваша основна зайнятість в ІТ зараз...':'employment_status',
 'Вкажіть вашу спеціалізацію Game Design': 'game_design_position'}

data.rename(columns=columns_translation_dict,inplace=True)

# Salary

Drop no salary

In [304]:
data = data.drop(data[data['salary'].isna()].index)

Change salary to float

In [305]:
data['salary'] = [round(float(str(i).replace(',','.'))) if type(i)==str else i for i in data['salary'] ]

# Timestamp

Change Timastamp to datetime object

In [306]:
date_time_formats = ['%Y-%m-%d %H:%M:%S','%d/%m/%Y %H:%M','%m.%d.%Y %H:%M:%S','%m.%d.%Y %H:%M','%d.%m.%Y %H:%M','%d/%m/%Y %H:%M','%m/%d/%Y %H:%M:%S']
stamps_to_date = []
for i in data['Timestamp']:
    stamp = ''
    if type(i) == str:
        for j in date_time_formats:
                try:
                    stamp = datetime.strptime(i,j)
                except:
                    pass
        stamps_to_date.append(stamp)
    else:
        stamps_to_date.append(i)

In [307]:
data['Timestamp'] = stamps_to_date

# Common columns

Find common subset of columns for all years by calculating the percentage of Nones in each column and each year. 

In [308]:
def get_common_columns(data):
    """
    Finds the common subset of columns for all years in data 'Timestamp' column 
    by calculating the percentange of Nones for each column and each year

    Parameters
    ----------
    - data (pd DataFrame): The DataFrame containing 'Timestamp' column and columns to be processed
    Returns
    -------
    - list: A list containing the common columns for all years
    """
    common_columns = []
    for column in data.columns:
        is_common_column = True
        years = data['Timestamp'].dt.year.unique()
        for year in years:
            year_data = data[data['Timestamp'].dt.year == year] # get data for the year
            year_nones_number = len(year_data[year_data[column].isna()]) # calculate the number of None for the year data
            year_nones_percentage = year_nones_number/len(year_data)
            if year_nones_percentage == 1: # if all values in year are None
                is_common_column = False # column is not common for all years
                break
        if is_common_column:
            common_columns.append(column)
    return common_columns

In [309]:
common_columns = get_common_columns(data)
common_columns

['Timestamp',
 'employment_status',
 'location',
 'gender',
 'age',
 'education',
 'english_level',
 'specialisation',
 'job_level',
 'job_type',
 'programming_language',
 'qa_position',
 'designer_position',
 'marketing_position',
 'hr_position',
 'management_position',
 'analyst_position',
 'data_scientist_position',
 'devops_position',
 'executive_position',
 'project_sphere',
 'company_type',
 'experience',
 'salary']

# Age

Change age to float

In [310]:
data['age'] = [round(float(str(i).replace(',','.'))) if type(i)==str else i for i in data['age'] ]

Remove people under age of 18

In [311]:
data.drop(data[data['age']<18].index,inplace=True)

# Gender

Convert to binary variable.

Values meaning:
 - 1 - man,
-  0 - woman

In [312]:
data['gender'] = [1 if i=='Чоловік' else 0 for i in data['gender'] ]

In [313]:
get_distribution_column(data['gender'])

,Values count,Values percentage
gender,,
1,59152,76.89
0,17780,23.11


# Location

Changing  values in location column to 'Not in Ukraine' for the people that are abroad because of War

In [314]:
abroad_index = data[(data['Де ви зараз живете? '].isin(['За кордоном (переїхали через війну, але планую повернутися)', 'За кордоном, але планую повернутися']))].index
data.loc[abroad_index,'location'] = 'Не Україна'

In [315]:
data['location'].unique()

array(['Київ', 'Дніпро', 'Львів', 'Харків', 'Інше', 'Одеса', 'Тернопіль',
       'Не Україна', 'Чернівці', 'Вінниця', 'Івано-Франківськ',
       'Запоріжжя', 'Миколаїв', 'Херсон', 'Житомир', 'Чернігів',
       'Кривий Ріг', 'Ужгород', 'Рівне', 'Хмельницький', 'Маріуполь',
       'Суми', 'Черкаси', 'Полтава', 'Луцьк', 'Кропивницький',
       'Івано-Франківськ чи область', 'Київ чи область',
       'Львів чи область', 'Житомир чи область', 'Дніпро чи область',
       'Тернопіль чи область', 'Вінниця чи область',
       'Кропивницький чи область', 'Чернігів чи область',
       'Харків чи область', 'Рівне чи область', 'Запоріжжя чи область',
       'Суми чи область', 'Чернівці чи область', 'Одеса чи область',
       'Черкаси чи область', 'Луцьк чи Волинська область',
       'Ужгород чи Закарпатська область', 'Полтава чи область',
       'Хмельницький чи область', nan, 'Миколаїв чи область',
       'Херсон чи область', 'Донецька чи Луганська область',
       'Не в Україні'], dtype=object)

Consolidation of location column values to one standart: mapping categories to administative centers of the regions

In [316]:
locations = [str(i) for i in data['location']]

# mapping dictionary: keys - categories that need to be changed,  values - categories to what need to be changed
cities_map = {                  
    'Маріуполь':'Донецька чи Луганська область',
    'Кривий Ріг':'Дніпро',
    'Донецька': 'Донецька чи Луганська область',
    'Не в Україні':'Не Україна',
    'За кордоном, але планую повернутися': 'За кордоном (переїхали через війну, але планую повернутися)'
}

for i,category in enumerate(locations):
    category_cleaned  = category.split(' чи')[0] # removing 'чи .... область' from values so that they correspond to one standart             
    if category_cleaned not in cities_map:
        if category != category_cleaned:
            cities_map[category] = category_cleaned # adding values with 'чи .... область' to mapping disctionary

data['location'] = data['location'].replace(cities_map) # mapping categories

data.drop(data[data['location'].isin(['Інше','Не Україна',np.nan])].index,inplace=True) # deleting needless values

In [317]:
get_distribution_column(data['location'])

,Values count,Values percentage
location,,
Київ,30853,42.80
Львів,12205,16.93
Харків,6304,8.75
Дніпро,4480,6.22
Одеса,3320,4.61
Вінниця,2320,3.22
Івано-Франківськ,1971,2.73
Черкаси,1081,1.50
Чернівці,986,1.37


Removing values covering less than 100 observations.

NOTE: THE AUTHOR BELIEVES THAT UNIFORM BORDERS OF UKRAINE ARE THOSE THAT WERE DETERMINED IN 1991 WHEN UKRAINE WAS DECLARED AN INDEPENDENT STATE AND INCLUDE ALL THE REGIONS, ADMINISTATIVE CENTERS OF WHICH ARE REPRESENTED ABOVE. CATEGORIES CONTAING A FEWER OBSERVATIONS THAN THE THRESHOLD WERE REMOVED ONLY FOR THE REASON OF BETTER MODEL PERFORMANCE AND  GENERALIZATION

In [318]:
delete_column_categories('location',threshold=100,data=data)

thanslating the cities names to English

In [319]:
location_translation_dict = {
    'Київ': 'Kyiv',
    'Дніпро': 'Dnipro',
    'Львів': 'Lviv',
    'Харків': 'Kharkiv',
    'Одеса': 'Odesa',
    'Тернопіль': 'Ternopil',
    'Чернівці': 'Chernivtsi',
    'Вінниця': 'Vinnytsia',
    'Івано-Франківськ': 'Ivano-Frankivsk',
    'Миколаїв': 'Mykolaiv',
    'Херсон': 'Kherson',
    'Житомир': 'Zhytomyr',
    'Чернігів': 'Chernihiv',
    'Запоріжжя': 'Zaporizhzhia',
    'Рівне': 'Rivne',
    'Хмельницький': 'Khmelnytskyi',
    'Суми': 'Sumy',
    'Черкаси': 'Cherkasy',
    'Луцьк': 'Lutsk',
    'Полтава': 'Poltava',
    'Кропивницький': 'Kropyvnytskyi',
    'Ужгород': 'Uzhhorod'
}

data['location'] = data['location'].replace(location_translation_dict) 

# Education

Changing values in education column to scientific degree for those people who have scientific degree

In [320]:
phd_index = data[data['Чи є у вас науковий ступінь? '].isin(['Кандидат наук / PhD', 'Доктор наук', 'Кандидат наук'])].index
data.loc[phd_index,'education']='Науковий ступінь (кандидат / доктор наук, PhD)'

Consolidating column categories to one standart

In [321]:
education_map= {
    'Незакінчена вища': 'Ще студент',
    'Ще студент вишу': 'Ще студент',
    'Вища (бакалавр, спеціаліст, магістр) - одна або декілька':'Вища',
    'Вища (бакалавр, спеціаліст, магістр)':'Вища',
    'Дві вищі': 'Вища',
    'Дві вищих': 'Вища',
    'Кандидат / доктор наук': 'Науковий ступінь (кандидат / доктор наук, PhD)',
    'Кандидат наук' : 'Науковий ступінь (кандидат / доктор наук, PhD)'
}
data['education'] =  data['education'].replace(education_map)
data = data.drop(columns='Чи є у вас науковий ступінь? ') # removing unnecessary column 

Translatiing education categories to English

In [322]:
education_translation_dict = {
    'Вища': 'higher',
    'Середня спеціальна': 'secondary_vocational',
    'Середня': 'secondary',
    'Науковий ступінь (кандидат / доктор наук, PhD)': 'scientific_degree',
    'Ще студент':'student'
}
data['education'] = data['education'].replace(education_translation_dict) 

In [323]:
get_distribution_column(data['education'])

,Values count,Values percentage
education,,
higher,62318,86.57
student,5065,7.04
secondary,2294,3.19
secondary_vocational,1184,1.64
scientific_degree,1124,1.56


# Specialisation

In [324]:
specialisation_map,undefined = make_map_from_similar_sets(data['specialisation'].unique())
specialisation_map,undefined

({'ERP / CRM': ['ERP/CRM'],
  'Game Design (Game, Level, Narrative)': ['Design (Game, Level, Narrative)'],
  'QA / AQA Engineer (Junior, Middle, Senior, Team/Tech Lead, Manager)': ['QA Engineer (Junior, Middle, Senior, Team/Tech Lead, Manager)',
   'QA Engineer (Junior, Middle, Senior, Team / Tech Lead, Manager)'],
  'HR/Recruiter/Learning and Development': ['HR', 'HR/Recruiter'],
  'Analyst (Business, Data, System etc)': ['Analyst (Business, Data, System)'],
  'Project/Product/Program/Delivery/Engineering Manager, Product Owner, Producer, Scrum Master': ['Project / Product / Program Manager, Product Owner, Scrum Master, Delivery Manager',
   'Project/Product/Program Manager, Product Owner, Scrum Master, Delivery Manager'],
  'Marketing, PR, SEO, Copywriter': ['Marketing, SEO, Copywriter'],
  'DevOps, SysAdmin': ['SysAdmin'],
  'Designer / Artist (Graphic, UX/UI, Game, Artist, Sound etc.)': ['Designer / Artist (Graphic, Video, UX/UI)',
   'Designer',
   'Sound Specialist / Designer'],


Modify specialisation map

In [325]:
specialisation_map['DevOps, SysAdmin, SRE'] = specialisation_map['DevOps, SysAdmin']
specialisation_map['DevOps, SysAdmin, SRE'].append('DevOps, SRE')
specialisation_map['DevOps, SysAdmin, SRE'].append('DevOps, SysAdmin')
del(specialisation_map['DevOps, SysAdmin'])
del(undefined['DevOps, SRE'])

specialisation_map[ 'Designer / Artist (Graphic, UX/UI, Game, Artist, Sound etc.)'] += specialisation_map['Game Design (Game, Level, Narrative)']
specialisation_map[ 'Designer / Artist (Graphic, UX/UI, Game, Artist, Sound etc.)'].append('Design (Game, Level, Narrative)')
specialisation_map['Designer / Artist (Graphic, UX/UI, Game, Artist, Sound etc.)']+=['Game Design (Game, Level, Narrative)','Design (Game, Level, Narrative)']
del(specialisation_map['Game Design (Game, Level, Narrative)'])

specialisation_map['Support (Customer, Technical, Community)'] =  ['Support','Customer Success','Technical writer','support']

specialisation_map['Software Engineer (Junior, Middle, Senior, Team / Tech Lead, Architect)'].append('Game Developer / Programmer')
del(undefined['Game Developer / Programmer'])
del(undefined['Sales / Business Development'])

specialisation_map['Sales / Business Development'] =  ['Sales Manager']



For sales manager define managment position Sales Manager

In [326]:
idx = data[data['specialisation']=='Sales Manager'].index
data.loc[idx,'management_position'] = 'Sales Manager'

Perform specialisation mapping

In [327]:
specialisation_map = {i:key for key,value in specialisation_map.items() for i in value}
data['specialisation'] =  data['specialisation'].replace(specialisation_map)

In [328]:
get_distribution_column(data['specialisation'])

,Values count,Values percentage
specialisation,,
"Software Engineer (Junior, Middle, Senior, Team / Tech Lead, Architect)",37090,51.52
"QA / AQA Engineer (Junior, Middle, Senior, Team/Tech Lead, Manager)",12569,17.46
"Project/Product/Program/Delivery/Engineering Manager, Product Owner, Producer, Scrum Master",4577,6.36
HR/Recruiter/Learning and Development,3123,4.34
"DevOps, SysAdmin, SRE",3051,4.24
"Analyst (Business, Data, System etc)",2588,3.60
"Designer / Artist (Graphic, UX/UI, Game, Artist, Sound etc.)",2308,3.21
"Data Science, Machine Learning, AI, Big Data, Data Engineer",1575,2.19
"Marketing, PR, SEO, Copywriter",1492,2.07


Remove categories with few observations

In [329]:
delete_column_categories('specialisation',100,data)

In [330]:
spec_rename_dct = {
    'СTO, Director of Engineering, Program Director, COO, CEO, (co-)founder': 'executive',
    'Software Engineer (Junior, Middle, Senior, Team / Tech Lead, Architect)': 'programming',
    'QA / AQA Engineer (Junior, Middle, Senior, Team/Tech Lead, Manager)':'qa',
    'Project/Product/Program/Delivery/Engineering Manager, Product Owner, Producer, Scrum Master': 'management',
    'HR/Recruiter/Learning and Development': 'hr',
    'DevOps, SysAdmin, SRE': 'operations',
    'Analyst (Business, Data, System etc)': 'analysis',
    'Designer / Artist (Graphic, UX/UI, Game, Artist, Sound etc.)': 'design',
    'Marketing, PR, SEO, Copywriter': 'marketing',
    'Support (Customer, Technical, Community)': 'support',
    'Sales / Business Development': 'development',
    'HTML Coder': 'html',
    'Security Specialist': 'security',
    'ERP / CRM': 'enterprise_systems',
    'DBA': 'dba',
    'Data Science, Machine Learning, AI, Big Data, Data Engineer':'data_science'

}
data['specialisation'] =  data['specialisation'].replace(spec_rename_dct)

# Experience

Mapping experience to numerical form

In [331]:
experience_map = {
    'Менше як 3 місяці': '0,1',
    'Пів року': '0,5',
    '3 місяці': '0,25',

}

# remove text parts
for i in  data['experience'].unique():
    if type(i)==str and i not in experience_map and (i.endswith('років') or i.endswith('рік') or i.endswith('роки') or i.endswith('року')):
        experience_map[i] = i.split(' ')[0]

# map values to one standart
data['experience'] =  data['experience'].replace(experience_map)

# change to float type
data['experience'] = [float(str(i).replace(',','.')) if type(i)==str else i for i in data['experience'] ]

In [332]:
data['experience'].unique()

array([ 8.  , 10.  ,  5.  ,  4.  ,  3.  ,  7.  ,  9.  ,  0.1 ,  2.  ,
        0.5 ,  6.  ,  1.  ,  0.25,  1.5 , 11.  , 15.  , 12.  , 14.  ,
       13.  ])

# English

In [333]:
get_distribution_column(data['english_level'])

,Values count,Values percentage
english_level,,
Upper-Intermediate,29383,40.92
Intermediate,24198,33.70
Advanced,8649,12.04
Pre-Intermediate,7951,11.07
Elementary,1612,2.24
Не знаю взагалі,19,0.03


Removing categories with few observations

In [334]:
delete_column_categories('english_level',100,data)

Renaming English level columns

In [335]:
eng_rename_dct ={
    'Upper-Intermediate': 'b2',
    'Intermediate':'b1',
    'Advanced': 'c1',
    'Pre-Intermediate': 'a2',
    'Elementary': 'a1'

}
data['english_level'] =  data['english_level'].replace(eng_rename_dct)

# Job level

In [336]:
data['job_level'].unique()

array([nan, 'Senior', 'Middle', 'Junior', 'Tech Lead', 'Intern/Trainee',
       'Head', 'Team Lead', 'Architect', 'Manager', 'Intern / Trainee',
       'Немає тайтлу'], dtype=object)

Mapping job level to one standart

In [337]:
job_level_mapping = {
    'Intern/Trainee':'trainee/intern',
    'Intern / Trainee':'trainee/intern',
     np.nan:'no_title',
    'Немає тайтлу':'no_title'
    }

In [338]:
data['job_level'] =  data['job_level'].replace(job_level_mapping)
data['job_level'] =  data['job_level'].replace({i:i.lower().replace(' ','_') for i in data['job_level'].dropna().unique()})

# Management

In [339]:
data['management_position'].unique()

array([nan, 'Delivery Manager', 'Product Manager', 'Program Manager',
       'Product Owner', 'Project Manager', 'Sales Manager',
       'Scrum Master', 'Business Manager', 'Engineering Manager', 'Інше',
       'Producer / General Producer', 'Art Producer', 'Agile Coach',
       'Delivery manager', 'ВА/РО/інжинірінг менеджер', 'Release Manager',
       'Compliance Project Manager Team Lead', 'IT Project Coordinator ',
       'Delivery Coordinator', 'Head of Cloud',
       'Business Analyst + Project Manager (як BA досліджую проєкт і описую вимоги, розробку яких потім і сам буду вести як PM)',
       'Head of department', 'Site Manager', 'Technical Product Owner',
       'Head of PMO ', 'Account Manager ', 'Head of Product',
       'Instructional designer ', 'Payments Technical Manager',
       'Senior project manager ', 'Team coordinator', 'Delivery Director',
       'Development manager ', 'Design manager', 'Program manager ',
       'Project coordinator', 'Operations manager ', 'PMO'

Finding map for similar values

In [340]:
management_map,management_undefined = make_map_from_similar_sets(data['management_position'].dropna().unique(),score_cutoff=95)
management_map,management_undefined

({'IT Project Coordinator ': ['Project coordinator'],
  ' Operations Manager': ['Operations Manager ', 'Operations manager '],
  'Program manager ': ['Program Manager'],
  'Delivery Manager': ['Delivery manager'],
  'Senior project manager ': ['Project Manager'],
  'Account Manager ': ['Account Manager'],
  'Head of Product Team': ['Head of Product']},
 {'Compliance Project Manager Team Lead': [],
  'Head of Development': [],
  'Agile Coach': [],
  'Product Manager': [],
  'PMO': [],
  'Business Analyst + Project Manager (як BA досліджую проєкт і описую вимоги, розробку яких потім і сам буду вести як PM)': [],
  'Design manager': [],
  'ВА/РО/інжинірінг менеджер': [],
  'Head of PMO ': [],
  'Assistant project manager ': [],
  'Інше': [],
  'Engineering Manager': [],
  'Development manager ': [],
  'Art Producer': [],
  'Site Manager': [],
  'Technical Product Owner': [],
  'Instructional designer ': [],
  'Release Manager': [],
  'Scrum Master': [],
  'Head of department': [],
  'Head

Mapping management values to one standart

In [341]:
# Making modification to management mapping dict
management_map['Project Manager'] = ['Senior project manager '] 
del(management_map['Senior project manager '])
# mapping
data['management_position'] = data['management_position'].replace({i:key for key,value in management_map.items() for i in value})


In [342]:
management_map

{'IT Project Coordinator ': ['Project coordinator'],
 ' Operations Manager': ['Operations Manager ', 'Operations manager '],
 'Program manager ': ['Program Manager'],
 'Delivery Manager': ['Delivery manager'],
 'Account Manager ': ['Account Manager'],
 'Head of Product Team': ['Head of Product'],
 'Project Manager': ['Senior project manager ']}

In [343]:
get_distribution_column(data['management_position'])

,Values count,Values percentage
management_position,,
Project Manager,2362,48.22
Product Manager,772,15.76
Delivery Manager,480,9.80
Product Owner,375,7.66
Sales Manager,321,6.55
Program manager,193,3.94
Scrum Master,193,3.94
Engineering Manager,87,1.78
Інше,32,0.65


Removing categories with few observations

In [344]:
delete_column_categories('management_position',100,data)

In [345]:
data['management_position'] = data['management_position'].replace({i:i[:-1].lower().replace(' ','_') if i.endswith(' ') else i.lower().replace(' ','_') for i in data['management_position'].dropna().unique()})

# HR

Finding map for similar values

In [346]:
hr_map,hr_undefined = make_map_from_similar_sets(data['hr_position'].dropna().unique(),score_cutoff=95)
hr_map,hr_undefined

({'HR Generalist ': ['HR Generalist'],
  'Talent Researcher / Sourcer': ['Talent Researcher/Sourcer'],
  'People Partner / Happiness Manager': ['People partner / Happiness manager'],
  'Recruitment Team Lead': ['Recruitment Lead'],
  'HR team lead ': ['HR Team Lead'],
  'HR Specialist ': ['HR specialist'],
  'Head of recruitment / CEO': ['Head of recruitment']},
 {'HRG рекрутер і hr одночасно ': [],
  'People partner /Recruiter': [],
  'Head of recruitment department ': [],
  'Talent Acquisition Manager': [],
  'HR Analyst': [],
  'HR Director': [],
  'Compensation and Benefit Specialist': [],
  ' Employee Benefits Team Lead': [],
  'Talent Acquisition Specialist': [],
  'Інше': [],
  'Resource Manager': [],
  'Learning and Development / Training and Development': [],
  'People and Culture Lead / HR Lead': [],
  'HR administrator ': [],
  'HR Business Partner': [],
  'Coordinator ': [],
  'English Teacher': [],
  'Recruiter': [],
  'IT Recruiter/HR Assistant': [],
  'HR Manager': [],
 

Mapping values to one standart

In [347]:
data['hr_position'] = data['hr_position'].replace({i:key for key,value in hr_map.items() for i in value})

In [348]:
get_distribution_column(data['hr_position'])

,Values count,Values percentage
hr_position,,
Recruiter,1159,37.11
HR Manager,717,22.96
HR Business Partner,243,7.78
HR Director,233,7.46
Talent Researcher / Sourcer,233,7.46
Talent Acquisition Manager,226,7.24
People Partner / Happiness Manager,155,4.96
Learning and Development / Training and Development,59,1.89
Інше,19,0.61


Removing categories with few observations

In [349]:
delete_column_categories('hr_position',100,data)

In [350]:
def rename_categories_to_variables(column_name,data):
    data[column_name] = data[column_name].replace({i:i[:-1].lower().replace(' / ','/').replace(' ','_') if i.endswith(' ') else i.lower().replace(' / ','/').replace(' ','_') for i in data[column_name].dropna().unique()})

In [351]:
data['hr_position'] = data['hr_position'].replace({i:i[:-1].lower().replace(' / ','/').replace(' ','_') if i.endswith(' ') else i.lower().replace(' / ','/').replace(' ','_') for i in data['hr_position'].dropna().unique()})

# Marketing

Getting marketing positions that are present in all years

In [352]:
get_intersaction_sets(get_ukr_unique('marketing_position',data))

{'Account Manager',
 'Chief Marketing Officer (CMO)',
 'Copywriter',
 'Digital Marketing Manager',
 'Email Marketing Specialist',
 'Employer Brand Manager',
 'Link Builder',
 'Marketing Manager',
 'Media Buyer',
 'PPC Specialist',
 'PR Manager',
 'Product Marketing Manager',
 'SEO Specialist',
 'SMM Manager',
 nan}

Remove all categories that are not present in all years

In [353]:
lst_other = []
for i in get_ukr_unique('marketing_position',data):
    lst_other+= set(i) - get_intersaction_sets(get_ukr_unique('marketing_position',data))
data.drop(data[data['marketing_position'].isin(set(lst_other))].index,inplace=True)

In [354]:
get_distribution_column(data['marketing_position'])

,Values count,Values percentage
marketing_position,,
Marketing Manager,200,15.65
Digital Marketing Manager,187,14.63
SEO Specialist,151,11.82
Copywriter,131,10.25
Employer Brand Manager,109,8.53
Chief Marketing Officer (CMO),101,7.90
PR Manager,82,6.42
PPC Specialist,72,5.63
SMM Manager,64,5.01


Remove categories with few observations

In [355]:
delete_column_categories('marketing_position',100,data)

In [356]:
rename_categories_to_variables('marketing_position',data)
data['marketing_position'] = data['marketing_position'].replace({'chief_marketing_officer_(cmo)':'chief_marketing_officer'})

# Data Science

In [357]:
get_distribution_column(data['data_scientist_position'])

,Values count,Values percentage
data_scientist_position,,
Data Engineer / Big Data Engineer,532,33.78
Data Scientist,517,32.83
Machine Learning Engineer,159,10.10
Machine / Deep Learning Engineer,87,5.52
Computer Vision Engineer,86,5.46
Research Engineer,84,5.33
BI Engineer,65,4.13
Big Data Engineer,26,1.65
Data Engineer,19,1.21


Finding map for similar values

In [358]:
ds_map,ds_undefined = make_map_from_similar_sets(data['data_scientist_position'].dropna().unique(),score_cutoff=90)
ds_map,ds_undefined

({'Machine / Deep Learning Engineer': ['Machine Learning Engineer'],
  'Data Engineer / Big Data Engineer': ['Data Engineer', 'Big Data Engineer']},
 {'Data Scientist': [],
  'BI Engineer': [],
  'Research Engineer': [],
  'Computer Vision Engineer': []})

Mapping similar values

In [359]:
data['data_scientist_position'] = data['data_scientist_position'].replace({i:key for key,value in ds_map.items() for i in value})

Removing categories with few observations

In [360]:
delete_column_categories('data_scientist_position',100,data)

In [361]:
rename_categories_to_variables('data_scientist_position',data)
data['data_scientist_position'] = data['data_scientist_position'].replace({'data_engineer/big_data_engineer':'data/big_data_engineer','machine/deep_learning_engineer':'ml/dl_engineer'})

# Designer

Examine  Game Design column

In [362]:
get_distribution_column(data['game_design_position'])

,Values count,Values percentage
game_design_position,,
Game Designer,59,48.36
Technical Designer,15,12.30
Level Designer,11,9.02
Narration Designer,8,6.56
Інше,7,5.74
Game Economy Designer,5,4.10
Quest Designer,3,2.46
System Game Designer,2,1.64
3d artist,2,1.64


Map values to one standart

In [363]:
designer_map,designer_undefined = make_map_from_similar_sets(data['game_design_position'].dropna().unique(),score_cutoff=95)
designer_map,designer_undefined

({'UI/UX Game Designer': ['UI/UX Designer', 'Game Designer'],
  '3D artist ': ['3d artist', '3D artist']},
 {'2D Marketing Artist': [],
  'Quest Designer': [],
  'Scriptwriter': [],
  'Game Analyst': [],
  'Narration Designer': [],
  '3D Prop Artist': [],
  'Metaverse design': [],
  'System Game Designer': [],
  'Інше': [],
  'Technical Designer': [],
  'Level Designer': [],
  'Game Economy Designer': [],
  'Designer Generalist': []})

In [364]:
designer_map[ '3D artist '].append('3D Prop Artist')
designer_map['UI/UX Designer']= ['UI/UX Game Designer']
del(designer_map['UI/UX Game Designer'])
data['game_design_position'] = data['game_design_position'].replace({i:key for key,value in designer_map.items() for i in value})

Coneect two columns (desiner_position, and game design ) into one

In [365]:
design_columns = ['designer_position',
                  'game_design_position']
for i in design_columns:
    for j in design_columns:
        if i!= j and data[(data[i].notna()) & (data[j].notna())].shape[0]!=0:
            print('Columns intersect')


for i,design_col in enumerate(design_columns):
    design_values_index =  data[design_col].dropna().index.tolist()
    design_values = data[design_col][ design_values_index].tolist()
    data.loc[ design_values_index,'designer_position'] =  design_values


In [366]:
designer_map,designer_undefined =  make_map_from_similar_sets(data['designer_position'].dropna().unique(),score_cutoff=95)
designer_map,designer_undefined

({'Concept Artist': ['Concept artist'],
  'UI / UX Designer': ['UI/UX Designer'],
  'Tech Game Designer': ['Game Designer'],
  '3D artist ': ['3D Artist']},
 {'Character Artist': [],
  'Graphic Designer': [],
  '2D Marketing Artist': [],
  'Scriptwriter': [],
  'Vehicle Artist': [],
  'Animator': [],
  'Narration Designer': [],
  'System Game Designer': [],
  'Illustrator': [],
  'Environment Artist': [],
  'Designer': [],
  'Other': [],
  'VFX Artist': [],
  'Інше': [],
  'Technical Designer': [],
  'Product Designer': [],
  'UI/UX Artist': [],
  'Sound Designer': [],
  'Designer Generalist': [],
  'UI/UX Writer': [],
  'Quest Designer': [],
  'Metaverse design': [],
  'Graphic / Video Designer': [],
  'Level Designer': [],
  'Web Designer': [],
  'Art Director': [],
  'Generalist Artist': [],
  'Game Analyst': [],
  'Creative Director': [],
  'Technical Artist': [],
  '2D Artist': [],
  'Game Economy Designer': [],
  'UI/UX Researcher': [],
  'Hard Surface Artist': []})

In [367]:
del(designer_map['Tech Game Designer'])

designer_map['Graphic / Video Designer'] = ['Graphic Designer']
designer_map['Інше'] = ['Other']
data['designer_position'] = data['designer_position'].replace({i:key for key,value in designer_map.items() for i in value})

In [368]:
get_distribution_column(data['designer_position'])

,Values count,Values percentage
designer_position,,
UI / UX Designer,940,40.83
Product Designer,339,14.73
3D artist,159,6.91
Graphic / Video Designer,134,5.82
2D Artist,122,5.30
Game Designer,116,5.04
Web Designer,77,3.34
Designer,70,3.04
Animator,64,2.78


Remove values with few observations

In [369]:
delete_column_categories('designer_position',100,data)

In [370]:
rename_categories_to_variables('designer_position',data)

# Company

In [371]:
get_distribution_column(data['company_type'])

,Values count,Values percentage
company_type,,
Аутсорсингова,30209,43.10
Продуктова,26148,37.30
Аутстафінгова,8716,12.43
Стартап,2769,3.95
Інша,1832,2.61
Фріланс,259,0.37
Державне підприємство / установа,163,0.23


remove observations that do not correspond to freelance company

In [372]:
data.drop(data[(data['company_type'] == 'Фріланс') & (~data['employment_status'].isin(['Я фрилансер(-ка) в ІТ','Так, я фрилансер']))].index,inplace=True)

map values to one standart

In [373]:
company_map = {'Фріланс' : 'Інша',
'Державне підприємство / установа' : 'Інша'}
data['company_type'] = data['company_type'].replace(company_map)

transalte values to English

In [374]:
company_type_translation_dct = {
    'Аутсорсингова': 'outsourcing',
    'Продуктова': 'product',
    'Інша':'other',
    'Стартап': 'startup',
    'Аутстафінгова': 'outstaffing'

}
data['company_type'] = data['company_type'].replace(company_type_translation_dct)

# Programming language

In [375]:
get_distribution_column(data['programming_language'])

,Values count,Values percentage
programming_language,,
JavaScript,9000,18.37
Java,7334,14.97
Python,5604,11.44
C# / .NET,5408,11.04
TypeScript,5229,10.67
...,...,...
ESQL,1,0.00
VBScript,1,0.00
BrightScript,1,0.00


Removing outliers

In [376]:
delete_column_categories('programming_language',10,data)

Mapping values to one standart

In [377]:
lang_map = {'C# / .NET': ['C#/.NET', 'C#'],
 'Мови розробки БД (PL/SQL, Transact-SQL)': ['SQL'],
 'Pascal/Delphi': ['Delphi'],
 'Salesforce Apex': ['Salesforce']}

data['programming_language'] = data['programming_language'].replace({i:key for key,value in lang_map.items() for i in value})
data['programming_language'].replace({'Іншу':'Інше','Salesforce Apex':'Інше','Інша мова': 'Інше'},inplace=True)

Removing categories with few observations

In [378]:
delete_column_categories('programming_language',100,data)

In [379]:
data.drop(data[data['programming_language'].isin(['Не можу обрати одну основну мову'])].index,inplace=True)

In [380]:
data['programming_language'] = data['programming_language'].replace({'Не можу обрати одну основну мову':'no_main_language','Не використовую':'not_using', 'Інше': 'other','Мови розробки БД (PL/SQL, Transact-SQL)':'db_languages'})

# QA

In [381]:
get_distribution_column(data['qa_position'])

,Values count,Values percentage
qa_position,,
Manual QA,7318,58.42
Automation QA,2946,23.52
General QA,2129,17.00
Performance QA,62,0.49
Embedded QA,45,0.36
Data Quality Engineer,26,0.21


Removing categories with few observations

In [382]:
delete_column_categories('qa_position',100,data)

In [383]:
data['qa_position'] = data['qa_position'].replace({i:i.strip(' QA').lower() for i in data['qa_position'].dropna().unique()})

# Director

In [384]:
get_distribution_column(data['executive_position'])

,Values count,Values percentage
executive_position,,
CTO,311,44.43
Director of Engineering,201,28.71
"CEO, (co-) founder",87,12.43
Program Director,78,11.14
COO,23,3.29


Removing categories with few observations

In [385]:
delete_column_categories('executive_position',100,data)

In [386]:
rename_categories_to_variables('executive_position',data)

# Ваша посада DevOps, SRE

In [387]:
get_distribution_column(data['devops_position'])

,Values count,Values percentage
devops_position,,
DevOps Engineer,2144,81.55
System Administrator,308,11.72
Site Reliability Engineer,177,6.73


In [388]:
rename_categories_to_variables('devops_position',data)

# Project`s sphere

Transmorming lists of spheres Series to spheres Series

In [389]:
idx = data[data['project_sphere'].isna()].index  # treating missing values
data.loc[idx, 'project_sphere'] = 'Інша'

#fletenning the sphere column
spheres = data['project_sphere'].str.split(', ')
sphere_lst = [j for i in spheres for j in i]
sphere_df = pd.Series(sphere_lst).dropna()
sphere_distribution = get_distribution_column(sphere_df) # distribution of flattened sphere column
sphere_distribution

,Values count,Values percentage
E-commerce,14873,14.56
Інша,9654,9.45
Fintech,5417,5.30
Mobile,5130,5.02
Media,3619,3.54
...,...,...
агенство,1,0.00
Outstaffing IT,1,0.00
Агенція,1,0.00
Career industry,1,0.00


Mapping similar values for each part

In [390]:
sphere_map,undefined = make_map_from_similar_sets(sphere_df[sphere_df.isin(sphere_distribution[sphere_distribution['Values count']>2].index)].unique(),score_cutoff=90)
sphere_map

{'Sports': ['Sport'],
 'Real estate': ['Real Estate'],
 'Saas': ['SaaS'],
 'Email marketing': ['Marketing', 'marketing'],
 'Web': ['web'],
 'Adtech / Advertising': ['Adtech'],
 'Oil and Gas': ['Oil & Gas'],
 'Fintech / Banking / Capital Management': ['Fintech / Banking',
  'Banking',
  'Fintech',
  'Bank'],
 'Machine Learning': ['E-learning'],
 'Other': ['other'],
 'Entertainment ': ['Entertainment'],
 'Medtech / Healthcare': ['Medtech',
  'education',
  'Education',
  'Edtech',
  'Edtech / Education'],
 'Software development': ['Software'],
 'Public services / Government': ['Government'],
 'AR / VR': ['AR/VR'],
 'Healthcare ': ['Healthcare', 'HealthCare', 'healthcare', 'Health care']}

In [391]:
sphere_map['Medtech / Healthcare'] = ['Healthtech','Medtech','Healthcare ','HealthCare','Healthcare ', 'Healthcare', 'healthcare','Health care','Health',]
sphere_map['Edtech / Education'] = ['Edtech','Education','education', 'Education ']
sphere_map['Marketing'] = ['marketing','Email marketing']

sphere_map['Health & Fitness'] = ['Fitness','Sports','Sport']
del(sphere_map['Machine Learning'])
del(sphere_map['Healthcare '])
del(sphere_map['Sports'])
del(sphere_map['Email marketing'])


sphere_map

{'Real estate': ['Real Estate'],
 'Saas': ['SaaS'],
 'Web': ['web'],
 'Adtech / Advertising': ['Adtech'],
 'Oil and Gas': ['Oil & Gas'],
 'Fintech / Banking / Capital Management': ['Fintech / Banking',
  'Banking',
  'Fintech',
  'Bank'],
 'Other': ['other'],
 'Entertainment ': ['Entertainment'],
 'Medtech / Healthcare': ['Healthtech',
  'Medtech',
  'Healthcare ',
  'HealthCare',
  'Healthcare ',
  'Healthcare',
  'healthcare',
  'Health care',
  'Health'],
 'Software development': ['Software'],
 'Public services / Government': ['Government'],
 'AR / VR': ['AR/VR'],
 'Edtech / Education': ['Edtech', 'Education', 'education', 'Education '],
 'Marketing': ['marketing', 'Email marketing'],
 'Health & Fitness': ['Fitness', 'Sports', 'Sport']}

In [392]:
spheres,sphere_distribution = replace_spheres(sphere_map,sphere_df,spheres)
sphere_distribution

,Values count,Values percentage
E-commerce,14873,14.56
Fintech / Banking / Capital Management,10272,10.06
Інша,9654,9.45
Mobile,5130,5.02
Medtech / Healthcare,5025,4.92
...,...,...
SAP,1,0.00
Oracle,1,0.00
Hr,1,0.00
sport,1,0.00


In [393]:
sphere_map,undefined = make_map_from_similar_sets(sphere_df[sphere_df.isin(sphere_distribution[sphere_distribution['Values count']>1].index)].unique(),score_cutoff=95)
sphere_map['Human Resources '] = ['hr','HR']
sphere_map

{'Supply chain': ['Supply Chain'],
 'betting': ['Betting'],
 'Marketing ': ['Marketing'],
 'E-learning': ['Learning', 'E-Learning'],
 'saas': ['SAAS', 'Saas'],
 'Medicine ': ['Medicine'],
 'Software Development': ['Software development'],
 'CRM': ['crm'],
 'Health and Fitness': ['Health & Fitness'],
 'Foodtech': ['FoodTech'],
 'Oil and gas': ['Oil and Gas'],
 'Hosting': ['hosting'],
 'Sales': ['sales'],
 'Human Resources ': ['hr', 'HR']}

In [394]:
spheres,sphere_distribution = replace_spheres(sphere_map,sphere_df,spheres)
sphere_distribution

,Values count,Values percentage
E-commerce,14873,14.56
Fintech / Banking / Capital Management,10272,10.06
Інша,9654,9.45
Mobile,5130,5.02
Medtech / Healthcare,5025,4.92
...,...,...
Medicines,1,0.00
SQL,1,0.00
SAP,1,0.00
Oracle,1,0.00


In [395]:
sphere_map,undefined  = make_map_from_similar_sets(sphere_df[sphere_df.isin(sphere_distribution.index)].unique(),score_cutoff=95)
sphere_map

{'Photo-editing': ['PhotoEditing'],
 'Outsourcing ': ['Outsourcing', 'outsourcing'],
 'AI/Machine learning': ['Machine Learning'],
 'E-commerce': ['eCommerce'],
 'service IT company': ['Service company'],
 'Disaster recovery': ['Disaster Recovery', 'disaster recovery'],
 'Field Services': ['Field Service'],
 'Sports and Betting': ['Sports betting'],
 'It support': ['Support', 'L2 Support', 'Support ', 'support'],
 'Web development studio': ['Development ', 'Web development'],
 'Health&wellness': ['health&wellness'],
 'Outsource ': ['Outsource'],
 'Pharmacy': ['pharmacy'],
 'Biotech': ['BioTech'],
 'Procurement ': ['procurement'],
 'Advertising': ['advertising'],
 'Site builder': ['Site Builder'],
 'E-health': ['Health '],
 'web analytics': ['Analytics'],
 ' Foodtech': ['Foodtech'],
 'digital marketing': ['Digital marketing'],
 'HR software': ['Software ', 'software'],
 'Recruiting ': ['Recruiting'],
 'Web/mobile ': ['Web&Mobile'],
 'Time tracking': ['Time-tracking'],
 'website builder'

In [396]:
sphere_map['Edtech / Education'] = sphere_map['elearning ']
sphere_map['Edtech / Education'].append('elearning ')
sphere_map['Public services / Government'] =  sphere_map['E-government ']
sphere_map['Public services / Government'].append('E-government ')
sphere_map['Web development'] = ['Web development studio']

In [397]:
spheres,sphere_distribution = replace_spheres(sphere_map,sphere_df,spheres)
sphere_distribution

,Values count,Values percentage
E-commerce,14874,14.56
Fintech / Banking / Capital Management,10272,10.06
Інша,9654,9.45
Mobile,5130,5.02
Medtech / Healthcare,5025,4.92
...,...,...
Holding,1,0.00
sport,1,0.00
Hr,1,0.00
Oracle,1,0.00


In [398]:
sphere_map = {'Marketing/Ads' : 'Adtech / Advertising',	
  'e-learning': 'Edtech / Education',
 'Health and Fitness': 'Medtech / Healthcare',
 'Health and fitness': 'Medtech / Healthcare',
 'E-government ': 'Public services / Government',
 'Finance ':  "Fintech / Banking / Capital Management",
 'Betting': 'Gambling',
 'Business Intelligence ': 'Big Data',
 'sales': "Retail",
 'bank': "Fintech / Banking / Capital Management",
 'Social Networks ': 'Media',
 'Medical': 'Medtech / Healthcare',
 'IoT': "Internet of Things",
 'NDA': "Legal",
 'Pharmacy': "Medtech / Healthcare",
 'Helthcare': "Medtech / Healthcare",
 'Health&wellness':"Medtech / Healthcare",
 'Social media': 'Media',
 'advertising': 'Adtech / Advertising',
 'Publishing': 'Media',
 'банк': "Fintech / Banking / Capital Management",
 'PropTech': "Real Estate",
 'Edtech': 'Edtech / Education',
 'GameDevнн':'GameDev',
 'Fintech / Banking': 'Fintech / Banking / Capital Management',
  'Adtech' :'Adtech / Advertising',
  'Fintech': 'Fintech / Banking / Capital Management',
  'Medtech': 'Medtech / Healthcare',
}

spheres,sphere_distribution = replace_spheres(sphere_map,sphere_df,spheres,reverse_dict=False)
sphere_distribution

,Values count,Values percentage
E-commerce,14874,14.56
Fintech / Banking / Capital Management,10280,10.06
Інша,9654,9.45
Mobile,5130,5.02
Medtech / Healthcare,5058,4.95
...,...,...
Hr,1,0.00
Oracle,1,0.00
SAP,1,0.00
SQL,1,0.00


Mapping spheres with few observations to 'Other'

In [399]:
small_spheres = sphere_distribution[(sphere_distribution['Values count']<=100) ].index
for i in small_spheres:
    sphere_map[i] = 'Інша'
spheres,sphere_distribution = replace_spheres(sphere_map,sphere_df,spheres,reverse_dict=False)
sphere_distribution

,Values count,Values percentage
E-commerce,14874,14.56
Інша,10690,10.47
Fintech / Banking / Capital Management,10280,10.06
Mobile,5130,5.02
Medtech / Healthcare,5058,4.95
Media,3627,3.55
GameDev,3509,3.44
Gambling,3237,3.17
Logistics,2953,2.89
Telecom,2918,2.86


In [400]:
spheres,sphere_distribution = replace_spheres({'Інша':'Other','Fintech / Banking / Capital Management':'Fintech/Banking'},sphere_df,spheres,reverse_dict=False)

In [401]:
spheres,sphere_distribution = replace_spheres({i: i.lower().replace(' / ','/').replace(' ','_') for i in sphere_distribution.index},sphere_df,spheres,reverse_dict=False)
sphere_distribution

,Values count,Values percentage
e-commerce,14874,14.56
other,10690,10.47
fintech/banking,10280,10.06
mobile,5130,5.02
medtech/healthcare,5058,4.95
media,3627,3.55
gamedev,3509,3.44
gambling,3237,3.17
logistics,2953,2.89
telecom,2918,2.86


In [402]:
data['project_sphere'] = spheres

# Job type

In [403]:
def remove_spec_outliers(spec,threshold):
    
    filtered_data = data[data['specialisation'] == spec]
    job_type_distribution = get_distribution_column(filtered_data['job_type'])
    small_values = job_type_distribution[job_type_distribution['Values count']<threshold].index
    data.drop(data[(data['specialisation'] == spec) & (data['job_type'].isin(small_values))].index,inplace=True)

In [404]:
job_type = get_distribution_column(data['job_type'])
job_type[job_type['Values count']>50]

,Values count,Values percentage
job_type,,
Back-end розробка,10498,23.69
Front-end розробка,5776,13.03
Full Stack розробка,5497,12.40
Back-end,4753,10.73
QA,3998,9.02
Front-end,2483,5.60
Mobile розробка,2476,5.59
"Робота з даними, аналіз даних",1891,4.27
Full Stack,1473,3.32


Perform specialisation mapping

In [405]:
dev_map ={
    'Game Development': ['Game develop',
   'GameDev',
   'gamedev',
   'Game developer',
   'Gamedev',
   'Game dev',
   'Unity',
   'Game Dev ? ',
   'GameDev Unity',
   'XR unity',
   'Game development',
   'Game Dev','Game Developer ','Game developer','GAME DEVELOPMENT','game development', 'Game development','game dev', 'Game dev', 'Game Dev'],

  'Automotive': ['Automotive ', 'Automotive', 'automotive','C++ automotive'],
  
 
  'Automation': [ 'automation'],

  'Cloud Infrastructure': ['Infrastructure '],
  'erp': ['ERP'],
  'Salesforce ': ['Salesforce'],

  'Generalist': ['generalist'],
  'blockchain': ['Blockchain'],
  'Rpa ': ['RPA'],

  '1С': ['1с'],
  'Computer Vision': ['Computer vision'],
  'embedded ': ['embedded', 'Embedded ', 'Embedded','Embedded SW','Embedded Linux', 'embedded ','Embedded'],
   'Front-end  розробка': ['Front-end','Front-end розробка'],
   'Back-end  розробка': ['Back-end'],
   'Desktop розробка': ['Desktop', 'desktop', 'Desktop розробка'],

  'Mobile  розробка': ['Mobile'],

  '1С': ['1с'],
 
 
  'System programming': ['System'],

   'Both Full-stack and mobile': ['Mobile and Full-stack',
  'Full-stack + mobile',
  'Full Stack + Mobile',
  'Both Full-stack and mobile'],

  'Mobile and front-end': ['Front-end/Mobile'],
  
  'Desktop розробка':['Desktop C++ Qt developer','Desktop Developer', 'Desktop developer','Desktop Apps','Desktop App','Desktop','desktop'],

  'DevOps/SRE': ['DevOps/SRE','DevOps, Systems Architect','DevOps, SRE','DevOps'],

  'Full Stack розробка': ['Full stack','Full stack ','Full Stack', 'Full stack','Full-stack','Full stack '],
  'QA': ['Automation QA','QA Automation'],
  'Embedded / IoT': ['Embedded','Embedded / IoT','Embedded / IoT', 'IoT, Embedded', 'Embedded/IoT','embeded','Embeded','Embedded '],
  'Інше': [np.nan, 'Робота з даними, аналіз даних']

}
data['job_type'] = data['job_type'].replace({i:key for key in dev_map for i in dev_map[key]})

Setting small values to 'Other'

In [406]:
job_type = get_distribution_column(data['job_type'])
small_values = job_type[job_type['Values count']<100].index
data['job_type'] = data['job_type'].replace(small_values, 'Інше')
get_distribution_column(data['job_type'])

,Values count,Values percentage
job_type,,
Інше,28307,41.04
Back-end розробка,15251,22.11
Full Stack розробка,8311,12.05
Front-end розробка,8261,11.98
QA,4000,5.80
Mobile розробка,3574,5.18
DevOps/SRE,1091,1.58
Embedded / IoT,184,0.27


Removing QA values where specialisation does not correspond job_type

In [407]:

job_type_qa = data[data['job_type'] == 'QA']
data.drop(job_type_qa[~(job_type_qa['specialisation'] == 'qa')].index,inplace=True)

not_qa = data[(data['specialisation'] == 'qa') & (~data['job_type'].isin( ['QA','Інше']))]
data.drop(not_qa.index, inplace=True )
data.loc[data[data['specialisation'] == 'qa'].index, 'job_type'] = 'QA'

data['job_type'] = data['job_type'].replace({'QA': np.nan}) 

Removing Devops values where specialisation does not correspond job_type

In [408]:
remove_spec_outliers('operations',30)

In [409]:
small_values = get_distribution_column(data[data['job_type'] == 'DevOps/SRE']['specialisation'])[get_distribution_column(data[data['job_type'] == 'DevOps/SRE']['specialisation'])['Values count']<50].index
data.drop(data[(data['job_type'] == 'DevOps/SRE') & (data['specialisation'].isin(small_values))].index,inplace=True)
data['job_type'] = data['job_type'].replace({'DevOps/SRE': np.nan})

Removing outliers in software engineers

In [412]:
data.drop(data[(data['specialisation'] == 'programming') & (data['job_type'].isin(['Інше']))].index,inplace=True)

Removing outliers for other specialisations

In [414]:
specs_to_clean_lst = [
    'dba',
    'security',
    'design',
    'analysis',
    'data_science',
    'support',
    'management',
    'operations'
]
for i in specs_to_clean_lst:
    remove_spec_outliers(i,100)

In [415]:
data.loc[data[data['specialisation'].isin(['executive','html','enterprise_systems'])].index,  'job_type'] = 'Інше'
data.loc[data[data['job_type']=='Інше'].index,'job_type' ] = np.nan

In [417]:
get_distribution_column(data['job_type'])

,Values count,Values percentage
job_type,,
Back-end розробка,14891,42.87
Front-end розробка,8093,23.30
Full Stack розробка,8036,23.14
Mobile розробка,3531,10.17
Embedded / IoT,183,0.53


# Full time

In [419]:
work_status = get_distribution_column(data['employment_status'])
work_status

,Values count,Values percentage
employment_status,,
"Так, працюю full-time",44143,66.30
Працюю full-time в ІТ-компанії чи ІТ-відділі,19485,29.26
Втратив(-ла) роботу в ІТ і шукаю нову,693,1.04
"Ні, тимчасово не працюю",517,0.78
"Так, працюю part-time",455,0.68
"Так, я фрилансер",388,0.58
Працюю part-time в ІТ-компанії чи ІТ-відділі,228,0.34
Я фрилансер(-ка) в ІТ,219,0.33
Зараз на повністю оплачуваному бенчі в ІТ-компанії,130,0.20


In [420]:
work_status_map = {
    'Працюю full-time в ІТ-компанії чи ІТ-відділі': 'Так, працюю full-time',
    'Зараз на повністю оплачуваному бенчі в ІТ-компанії': 'Так, працюю full-time',
    'Я фрилансер(-ка) в ІТ': 'Так, я фрилансер',
    'Працюю part-time в ІТ-компанії чи ІТ-відділі': 'Так, працюю part-time',
    'Зараз на частково оплачуваному бенчі в ІТ-компанії':'Так, працюю part-time'
}
data['employment_status'] = data['employment_status'].replace(work_status_map)
data = data[data['employment_status'] == 'Так, працюю full-time'].drop('employment_status',axis=1)

In [421]:
common_columns.remove('employment_status')

# New features engineering

Year variable engineering

In [422]:
data['year'] = data['Timestamp'].dt.year

War variable engineering

In [423]:
data['war'] = [ 1 if i >=  pd.to_datetime('2022-02-24 04:00:00') else 0 for i in data['Timestamp']]

Logarithm of experience variable engineering

In [424]:
data['log_experience'] = np.log(data['experience'])

Logarithm of age variable engineering

In [425]:
data['log_age'] = np.log(data['age'])

# Save cleaned data

In [426]:
data.to_csv('cleaned_data.csv')

In [427]:
for i in common_columns:
    if i not in ['Timestamp','salary','age','project_sphere','experience']:
        print(i)
        print(get_distribution_column(data[i]))

location
                 Values count  Values percentage
location                                        
Kyiv                    27125              42.54
Lviv                    10876              17.06
Kharkiv                  5747               9.01
Dnipro                   4011               6.29
Odesa                    2938               4.61
Vinnytsia                2103               3.30
Ivano-Frankivsk          1735               2.72
Cherkasy                  929               1.46
Chernivtsi                890               1.40
Ternopil                  812               1.27
Khmelnytskyi              807               1.27
Zaporizhzhia              736               1.15
Poltava                   701               1.10
Rivne                     654               1.03
Zhytomyr                  641               1.01
Uzhhorod                  623               0.98
Lutsk                     555               0.87
Sumy                      500               0.78
Mykolaiv   

----